<a href="https://colab.research.google.com/github/yoonkim313/dataCampusProject-Team10/blob/master/ocr/test_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This can be executed in https://colab.research.google.com "Python 3 / GPU" runtime.

In [ ]:
!git clone https://github.com/yoonkim313/dataCampusProject-Team10.git

모델 저장 경로 확인

In [4]:
%cd ./saved_models

/workspace/dataCampusProject-Team10/dataCampusProject-Team10/ocr/saved_models


In [5]:
models = {
    'best_accuracy.pth': 'https://drive.google.com/open?id=1JqMF8-FSgpAaSfhq-CFbERXpX5rTTQrW'}
for k, v in models.items():
    doc_id = v[v.find('=')+1:]
    !curl -c /tmp/cookies "https://drive.google.com/uc?export=download&id=$doc_id" > /tmp/intermezzo.html
    !curl -L -b /tmp/cookies "https://drive.google.com$(cat /tmp/intermezzo.html | grep -Po 'uc-download-link" [^>]* href="\K[^"]*' | sed 's/\&amp;/\&/g')" > $k

!ls -al *.pth

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3262    0  3262    0     0   4700      0 --:--:-- --:--:-- --:--:--  4693
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0    659      0 --:--:-- --:--:-- --:--:--   659
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0
100  244M    0  244M    0     0  3604k      0 --:--:--  0:01:09 --:--:-- 6221k
-rwxrwxrwx 1 1000 1000 255928887  8월 28  2020 best_accuracy.pth


evaluation data download

In [7]:
%cd ../data/data_lmdb/evaluation/

/workspace/dataCampusProject-Team10/dataCampusProject-Team10/ocr/data/data_lmdb/evaluation


In [ ]:
data = {
    'evaluation.mdb': 'https://drive.google.com/open?id=1JqMF8-FSgpAaSfhq-CFbERXpX5rTTQrW'}
for k, v in data.items():
    doc_id = v[v.find('=')+1:]
    !curl -c /tmp/cookies "https://drive.google.com/uc?export=download&id=$doc_id" > /tmp/intermezzo.html
    !curl -L -b /tmp/cookies "https://drive.google.com$(cat /tmp/intermezzo.html | grep -Po 'uc-download-link" [^>]* href="\K[^"]*' | sed 's/\&amp;/\&/g')" > $k

!ls -al *.pth

ocr 디렉토리로 이동

In [8]:
%cd ../../..

/workspace/dataCampusProject-Team10/dataCampusProject-Team10/ocr


In [ ]:
!CUDA_VISIBLE_DEVICES=0 python3 ./deep-text-recognition/test.py \
--eval_data ./data/data_lmdb/evaluation/ \
--benchmark_all_eval \
--Transformation TPS \
--FeatureExtraction VGG \
--SequenceModeling BiLSTM \
--Prediction Attn \
--saved_model ./saved_modesl/best_accuracy.pth

In [ ]:
output = !CUDA_VISIBLE_DEVICES=0 python3 ./deep-text-recognition/demo.py \
--benchmark_all_eval \
--Transformation TPS \
--FeatureExtraction VGG \
--SequenceModeling BiLSTM \
--Prediction Attn \
--image_folder ./text_detection/result/test1_structure/ \
--saved_model ./saved_models/best_accuracy.pth

In [ ]:
output

In [ ]:
from IPython.core.display import display, HTML
from PIL import Image
import base64
import io
import pandas as pd

data = pd.DataFrame()
for ind, row in enumerate(output[output.index('image_path \t predicted_labels \t confidence score')+2:]):
    row = row.split('\t')
    filename = row[0].strip()
    label = row[1].strip()
    conf = row[2].strip()
    img = Image.open(filename)
    img_buffer = io.BytesIO()
    img.save(img_buffer, format="PNG")
    imgStr = base64.b64encode(img_buffer.getvalue()).decode("utf-8")
    
    data.loc[ind, 'img'] = '<img src="data:image/png;base64,{0:s}">'.format(imgStr)
    data.loc[ind, 'id'] = filename
    data.loc[ind, 'label'] = label
    data.loc[ind, 'conf'] = conf
    
html_all = data.to_html(escape=False)
display(HTML(html_all))